## Backtesting 
### Métricas (2)
En este cuaderno continuamos con el cáclulo de métricas, pero introducimos
- el uso de reglas de trading que aplicamos a varios activos a la vez
- la comparación respecto al índice de referencia

De momento esta comparación no es justa porque no tenemos una correspondencia con el universo de acciones invertibles

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import itertools

### Datos 
Asumiremos por un tema se simplicidad en los ejemplos, que nuestro universo de acciones invertibles son las siguientes acciones del IBEX35.

In [ ]:
ticker_list = ['BBVA','SAN','REP','TEF','IBE','FER','ITX','ACS','GRF']

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
close_series = {ticker: df.close for ticker, df in stock_data.items()
                if ticker in ticker_list}
stock_df = pd.DataFrame(close_series)
stock_df = stock_df.loc['2010':]

In [ ]:
stock_df.head()

Eliminaremos primero las acciones que no han estado en el índice durante todo el periodo de evaluación

In [ ]:
stock_df.dropna(axis=1, how='all', inplace=True)
stock_df

También usaremos el **benchmark**, la  serie del ibex con dividendos

In [ ]:
with open('../data/benchmark.pkl', 'rb') as handle:
    benchmarks = pickle.load(handle)

In [ ]:
benchmarks.keys()

In [ ]:
benchmark = benchmarks['ibex_div'].close.loc['2010':]
benchmark.plot()

____
### Estrategia Media Móvil
Implementamos una estrategia de buscar las tendencias alcistas cuando
el precio esté por encima de la media móvil. La idea es tener una estrategia
sencilla independiente por cada acción

In [ ]:
def movaverage_states(vseries, win):
    sma = vseries.rolling(win).mean()
    signal_states = (vseries > sma).astype(float)
    trading_states = signal_states.shift(1)
    trading_states.iloc[0] = 0
    return trading_states

In [ ]:
def plot_sma_states(vseries, win):
    sma = vseries.rolling(win).mean()
    states = movaverage_states(vseries, win)
  
    fig, ax = plt.subplots(figsize=(20,4))
    vseries.plot(ax=ax)
    sma.plot(ax=ax)

    for i, idx in enumerate(states.index[:-1]):
        if states.loc[idx] == 1:
            ax.axvspan(idx, states.index[i+1], facecolor='g', alpha=0.2)

In [ ]:
plot_sma_states(stock_df['ITX'].iloc[:-500], 30)

In [ ]:
def state_return_serie(price, states):
    """ 
    """
    real_serie = price.dropna()
    ret = real_serie.pct_change()
    ret.iloc[0] = 0
    ret_series = ret * states
    return ret_series
    

___

In [ ]:
stock_states = stock_df.apply(movaverage_states, win=50)
stock_states

In [ ]:
example_returns = state_return_serie(stock_df['FER'], stock_states['FER'])
example_returns

solo tenemos fluctuación de retorno cuando estamos invertidos

In [ ]:
example_returns.plot()

____
Calculamos la serie de rendimientos a partir de los estados, para todas las acciones

In [ ]:
sma_returns = [state_return_serie(stock_df[tk], stock_states[tk]) for tk in stock_df.columns]
sma_returns_df = pd.concat(sma_returns, axis=1)
sma_returns_df

In [ ]:
sma_performance = (sma_returns_df + 1).cumprod()
sma_performance

In [ ]:
sma_performance[['IBE','REP','FER']].plot()

___

### Calculo agregado de rentabilidad
Si hacemos esta estrategia equiponderada tendriamos que dividir el resultado
entre todos los elementos que se encuentren en el índice ese día 

In [ ]:
sma_performance.iloc[0].dropna().count()

In [ ]:
def equiperformance(performance_row):
    n = performance_row.dropna().count()
    return performance_row/n

In [ ]:
equiperformance(sma_performance.iloc[0])

In [ ]:
equity_performance = sma_performance.apply(equiperformance, axis=1)
equity_performance

En realidad el rendimiento es el mismo, solo que aportará la parte proporcional al portfolio

In [ ]:
equity_performance[['IBE','REP','FER']].plot()

In [ ]:
porfolio_performance = equity_performance.sum(axis=1)
porfolio_performance.plot()

### Comparación con el Benchmark

In [ ]:
relative_bm = benchmark/benchmark.iloc[0]

In [ ]:
estrategias = pd.DataFrame({
    'SMA': porfolio_performance,
    'Ibexdiv': relative_bm

})

In [ ]:
estrategias.plot()

### Métricas respecto al Benchmark

Primero el **periodo de tiempo** de la evaluación

In [ ]:
init_date, end_date = relative_bm.index[0], relative_bm.index[-1]
years_in = (end_date - init_date) / pd.Timedelta(days=365, hours=6)
years_in

In [ ]:
bdays_year = int(relative_bm.shape[0]/years_in)
bdays_year

**Rendimiento Anualizado** 


In [ ]:
portval = porfolio_performance

In [ ]:
port_total_ret = portval.iloc[-1]/portval.iloc[0] - 1
port_total_ret

In [ ]:
ann_ret = np.power(port_total_ret + 1, 1/years_in) - 1
ann_ret

**Active_return**

In [ ]:
bench_ret = relative_bm.iloc[-1]/relative_bm.iloc[0] - 1
bench_annret = np.power(bench_ret + 1, 1/years_in) - 1
bench_annret

In [ ]:
active_ret = ann_ret - bench_annret
active_ret

**Information Ratio**

In [ ]:
logret = np.log(portval).diff().dropna()
bench_logret = np.log(relative_bm).diff().dropna()

In [ ]:
active_dayret = logret - bench_logret
active_dayret.head()

In [ ]:
tracking_error = np.sqrt(bdays_year) * active_dayret.std()
inform_ratio = (bdays_year * active_dayret.mean()) / tracking_error
inform_ratio

___